In [15]:
import numpy as np

In [2]:
! pip install gensim
! pip install ntlk
! pip install scikit-learn

ERROR: Could not find a version that satisfies the requirement ntlk (from versions: none)
ERROR: No matching distribution found for ntlk


In [3]:
from nltk.tokenize import sent_tokenize, word_tokenize
import gensim
from gensim.models import Word2Vec

In [4]:
from google.colab import files


uploaded = files.upload()

Saving singapore_airlines_reviews.csv to singapore_airlines_reviews (1).csv


In [7]:

import pandas as pd
import io

df = pd.read_csv(io.BytesIO(uploaded['singapore_airlines_reviews (1).csv']))
print(df)


                 published_date published_platform  rating    type  \
0     2024-03-12T14:41:14-04:00            Desktop       3  review   
1     2024-03-11T19:39:13-04:00            Desktop       5  review   
2     2024-03-11T12:20:23-04:00            Desktop       1  review   
3     2024-03-11T07:12:27-04:00            Desktop       5  review   
4     2024-03-10T05:34:18-04:00            Desktop       2  review   
...                         ...                ...     ...     ...   
9995  2018-08-06T03:48:21-04:00            Desktop       5  review   
9996  2018-08-05T22:50:29-04:00             Mobile       5  review   
9997  2018-08-05T22:47:06-04:00            Desktop       5  review   
9998  2018-08-05T20:32:03-04:00            Desktop       4  review   
9999  2018-08-05T20:19:51-04:00            Desktop       4  review   

                                                   text  \
0     We used this airline to go from Singapore to L...   
1     The service on Singapore Airlines S

In [8]:
# print(df["text"])
reviews = np.array(df["text"])
rating = np.array(df['rating'])
# print(reviews)
trainans = rating[:8000]
testans = rating[8000:]
# print(trainans)

In [9]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer()


tfidf_matrix = vectorizer.fit_transform(reviews)

tfidf_array = tfidf_matrix.toarray()
train = tfidf_array[:8000]
test = tfidf_array[8000:]
print(tfidf_array)



[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


In [14]:
from sklearn.linear_model import LinearRegression

model = LinearRegression()

model.fit(train[:8000], trainans[:8000])

ans_pred = model.predict(test)
near = 0
accurate_results = 0
for i in range(2000):
  a = ans_pred[i]
  if a >= 4.5:
    ans_pred[i] = 5
  elif a >= 3.5:
    ans_pred[i] = 4
  elif a >= 2.5:
    ans_pred[i] = 3
  elif a >= 1.5:
    ans_pred[i] = 2
  else:
    ans_pred[i] = 1
  if ans_pred[i] == testans[i]:
    accurate_results += 1
  if ans_pred[i] - testans[i] <= 1 and ans_pred[i] - testans[i] >= -1:
    near += 1
print(accurate_results)
print(near)

867


A linear regression model is clearly incapable of properly identifying the sentiments. However it still does a way better job than what a random classifier would have achieved. We get an accuracy of 43 percent here as oppposed to the expected random accuracy of 20 percent.

In [16]:
from sklearn.tree import DecisionTreeRegressor

model = DecisionTreeRegressor()

model.fit(train[:8000], trainans[:8000])

ans_pred = model.predict(test)
near  = 0
accurate_results = 0
for i in range(2000):
  a = ans_pred[i]
  if a >= 4.5:
    ans_pred[i] = 5
  elif a >= 3.5:
    ans_pred[i] = 4
  elif a >= 2.5:
    ans_pred[i] = 3
  elif a >= 1.5:
    ans_pred[i] = 2
  else:
    ans_pred[i] = 1
  if ans_pred[i] == testans[i]:
    accurate_results += 1
  if ans_pred[i] - testans[i] <= 1 and ans_pred[i] - testans[i] >= -1:
    near += 1
print(accurate_results)
print(near)

1037


With a way faster runtime, The Decision Tree Regressor achieved an accuracy of 52 percent.

In [18]:
from sklearn.linear_model import Lasso

model = Lasso(alpha = 0.1)

model.fit(train[:8000], trainans[:8000])

ans_pred = model.predict(test)
near = 0
accurate_results = 0
for i in range(2000):
  a = ans_pred[i]
  if a >= 4.5:
    ans_pred[i] = 5
  elif a >= 3.5:
    ans_pred[i] = 4
  elif a >= 2.5:
    ans_pred[i] = 3
  elif a >= 1.5:
    ans_pred[i] = 2
  else:
    ans_pred[i] = 1
  if ans_pred[i] == testans[i]:
    accurate_results += 1
  if ans_pred[i] - testans[i] <= 1 and ans_pred[i] - testans[i] >= -1:
    near += 1
print(accurate_results)
print(near)

491


In [22]:
from sklearn import linear_model

model = linear_model.BayesianRidge()

model.fit(train[:8000], trainans[:8000])

ans_pred = model.predict(test)
near = 0
accurate_results = 0
for i in range(2000):
  a = ans_pred[i]
  if a >= 4.5:
    ans_pred[i] = 5
  elif a >= 3.5:
    ans_pred[i] = 4
  elif a >= 2.5:
    ans_pred[i] = 3
  elif a >= 1.5:
    ans_pred[i] = 2
  else:
    ans_pred[i] = 1
  if ans_pred[i] == testans[i]:
    accurate_results += 1
  if ans_pred[i] - testans[i] <= 1 and ans_pred[i] - testans[i] >= -1:
    near += 1
print(accurate_results)
print(near)

1098
1896


As we can see, we achieve a staggering accuracy of 55 percent with a Bayesian Ridge Model. Moreover, of the rest, the ones which lie within +- 1 are: 95 percent. Reviews are in general hard to predict hence these numbers are in my opinion sufficient.